In [ ]:
from datascience import *
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import numpy as np

# Bootstrap

### Recap

[Government Compensation in California](https://publicpay.ca.gov/Reports/Counties/County.aspx?entityid=42&year=2020)
Let's look at how much Santa Barbara County employees got paid in 2020.

In [ ]:
sb_pop = Table.read_table('data/santabarbara-county-salaries-2020.csv').where('TotalWages', are.above(10000))
sb_pop = sb_pop.where('TotalWages', are.above(10*40*52))#.sort('TotalWages')
# show me the employees with the lowest compensation 
sb_pop.sort('TotalWages', descending = False).show(5)

In [ ]:
sb_pop.num_rows

In [ ]:
pop_median = percentile(50, sb_pop.column('TotalWages'))
sb_bins = np.arange(0, 500000, 25000)
sb_pop.hist('TotalWages', bins=sb_bins)
print("Population Median = ", pop_median)
plots.title('Population');

In [ ]:
# Random sample of size 300
our_sample = sb_pop.sample(300, with_replacement = False)
our_sample_median = percentile(50, our_sample.column('TotalWages'))
our_sample.hist('TotalWages', bins=sb_bins)
print("Population Median = ", pop_median)
print("Sample Median = ", our_sample_median)
plots.title('Our sample');

In [ ]:
# Empirical distribution of the sample median
# assuming we can just resample from the population
def one_sample_median():
    single_sample = sb_pop.sample(300, with_replacement = False)
    return percentile(50, single_sample.column('TotalWages'))

medians = make_array()
for i in np.arange(1000):
    new_median = one_sample_median()
    medians = np.append(medians, new_median)

In [ ]:
med_bins = np.arange(65000, 90000, 900)

Table().with_column(
    'Sample Medians', medians
).hist('Sample Medians', bins=med_bins)

plots.scatter(pop_median, 0, color="red");
plots.title('Sample Medians (1K Samples from Pop)');

### Bootstrap

In [ ]:
# Take a bootstrap (re)sample of size 300, WITH replacement
boot_sample = our_sample.sample(300, with_replacement=True)
boot_sample.hist('TotalWages', bins=sb_bins)
plots.title('1 Bootstrap sample');

print("Population Median =       ", pop_median)
print("Our Sample Median =       ", our_sample_median)
print("Bootstrap Sample Median = ", 
      percentile(50,boot_sample.column('TotalWages')))

In [ ]:
def one_bootstrap_median():
    single_sample = our_sample.sample()
    return percentile(50, single_sample.column('TotalWages'))

In [ ]:
# Bootstrap our sample 1000 times
bootstrap_medians = make_array()
for i in np.arange(1000):
    new_median = one_bootstrap_median()
    bootstrap_medians = np.append(bootstrap_medians, new_median)

In [ ]:
Table().with_column(
    'Bootstrap Medians', bootstrap_medians
).hist('Bootstrap Medians', bins=med_bins)

plots.scatter(pop_median, 0, color="red");
plots.scatter(our_sample_median, 0, color="blue");
plots.title('Bootstrap Medians (1K Bootstraps from our Sample)');

### 95% Confidence Interval

In [ ]:
# Make an interval based on the middle 95% of bootstrap samples

left = percentile(2.5, bootstrap_medians)
right = percentile(97.5, bootstrap_medians)

Table().with_column(
    'Bootstrap Medians', bootstrap_medians
).hist('Bootstrap Medians', bins=med_bins)

plots.plot([left, right], [0,0], color="gold",lw=4, zorder=1);
plots.scatter(pop_median, 0, color="red", zorder=2);
plots.scatter(our_sample_median, 0, color="blue", zorder=2);
plots.title('Bootstrap Medians (1K Bootstraps from our Sample)');

## Another Example: Mean Maternal Age

In [ ]:
# This time we have a sample, but no population data!
births = Table.read_table('data/baby.csv')
births.show(5)

In [ ]:
births.hist('Maternal Age')

In [ ]:
mean_age = np.mean(births.column('Maternal Age'))
mean_age

In [ ]:
def one_bootstrap_mean():
    return np.mean(births.sample().column('Maternal Age'))

In [ ]:
bootstrap_means = make_array()

for i in np.arange(1000):
    new_mean = one_bootstrap_mean()
    bootstrap_means = np.append(bootstrap_means, new_mean)
    
left = percentile(2.5, bootstrap_means)
right = percentile(97.5, bootstrap_means)

In [ ]:
Table().with_column('Bootstrap means', bootstrap_means).hist()

plots.plot([left,right], [0,0], color="gold", lw=3, zorder=1);
plots.scatter(mean_age,0,color="blue", zorder=2);
plots.title('Bootstrap Means (1K Bootstraps from our Sample)');